# Stock returns during easter

## Introduction
### Emperical strategy
We attempt to estimate the causal effects of easter on stockmarket returns using af Difference in Difference (DiD) identification strategy. We estimate the equation:

$$
\text{Daily\_return}_{it}=\beta_{0}+\beta_{1}\times\text{Chrisitan}_{it}+\beta_{2}\text{Easter}_{it}+\beta_{3}\text{Easter}_{it}\times\text{Christian}_{it}
$$

Under some assumptions we can we can derive $\beta_{3}$ to have a causal interpretation, where $\beta_{3}$ is given by: 
$$
\beta_{3}=E[\text{Daily\_return}_{it}\vert\text{Easter=1},\text{Christ}=1]-E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=1]
$$

The equation can be derived using the observed differences from period $t-1$ to period $t$ in a Christian and non Christian country. 
The oberved changes in the daily returns of a christian country in the given by $\Delta\text{Daily\_return}_{christ}=E[\text{Daily\_return}\vert\text{Easter=1},\text{Christ}=1]-E[\text{Daily\_return}_{it-1}\vert\text{Easter=0},\text{Christ}=1]$

We can add and subtract the counterfactual scenario of $E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=1]$

$$\Delta\text{Daily\_return}_{Christ}=E[\text{Daily\_return}_{it}\vert\text{Easter=1},\text{Christ}=1]-E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=1]$$
 
$$+E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=1]-E[\text{Daily\_return}_{it-1}\vert\text{Easter=0},\text{Christ}=1]$$
 
If we subtract the observed changes from period t-1 to t for a predominantly non Christian country 

$$ \Delta\text{Daily\_return}_{Christ}-\Delta\text{Daily\_return}_{nonChrist}=E[\text{Daily\_return}_{it}\vert\text{Easter=1},\text{Christ}=1]-E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=1] $$
$$ +E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=1]-E[\text{Daily\_return}_{it-1}\vert\text{Easter=1},\text{Christ}=1] $$

$$ -E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=0]-E[\text{Daily\_return}_{it-1}\vert\text{Easter=0},\text{Christ}=0] $$

Under the assumption that 
$$
E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=1]-E[\text{Daily\_return}_{it-1}\vert\text{Easter=1},\text{Christ}=1]=$$
$$E[\text{Daily\_return}_{it}\vert\text{Easter=0},\text{Christ}=0]-E[\text{Daily\_return}_{it-1}\vert\text{Easter=0},\text{Christ}=0]
$$

which is commonly known as the parralel trend assumption, we have identfied $\beta_{3}$ and can interpret the results from the regression as the causal effect of Easter on stock returns.

## Data
We use data on the placement of easter from the famous website ugenr.dk. We obtain the data for easter dates by "scraping" ugenr.dk for the easter dates. 

Imports and set magics:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
# from matplotlib_venn import venn2
import os
import datetime

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

# customized directory
input_dir = os.path.join(os.getcwd(),'data')


# Read and clean data

Import your data, either through an API or manually, and load it. 

### Easter dates

The purpose is to generate a dataset with all the easter week dates from 2000 to 2023 <br>
To do so, we scrape data from ugenr.dk <br>
[This](https://ugenr.dk/2023/påske) is an example of how the webpage would look for 2023.

In [2]:
# a. import raw data from scrape.
ugenr_scrape = pd.read_pickle(os.path.join(input_dir,'paaskedage.pkl')) 

# b. sort after year
ugenr_scrape.sort_values(by='year',inplace=True, ignore_index=True)

ugenr_scrape

,year,month,start_date,end_date,uge,dato
0,2000,april,17,23,uge 16,mandag 17. – søndag 23. april 2000
1,2001,april,9,15,uge 15,mandag 9. – søndag 15. april 2001
2,2002,marts,25,31,uge 13,mandag 25. – søndag 31. marts 2002
3,2003,april,14,20,uge 16,mandag 14. – søndag 20. april 2003
4,2004,april,5,11,uge 15,mandag 5. – søndag 11. april 2004
5,2005,marts,21,27,uge 12,mandag 21. – søndag 27. marts 2005
6,2006,april,10,16,uge 15,mandag 10. – søndag 16. april 2006
7,2007,april,2,8,uge 14,mandag 2. – søndag 8. april 2007
8,2008,marts,17,23,uge 12,mandag 17. – søndag 23. marts 2008
9,2009,april,6,12,uge 15,mandag 6. – søndag 12. april 2009


From the **dato**-variable, we can extract the month and when easter started and ended for each year. <br>
We want to convert this information to (daily) dates, i.e. 2022-04-11, 2022-04-12, ... <br>

We would expect the number of observations to be the number of years times number of days in easter week: <br>
$ 23 \cdot 7 = 161 $

In [4]:
# a. convert months to digits
month_dict = {'april' : '4', 'marts' : '5'}
ugenr_scrape.replace({"month": month_dict}, inplace = True)

# b. generate (daily) dates by iterating over rows
dates = []
for year, month, start in ugenr_scrape[['year', 'month', 'start_date']].itertuples(index=False):
    # 1. define first date of easter for given year
    first_date = datetime.date(int(year), int(month), int(start))

    # 2. generate list of all days in easter week
    date_int = [first_date + datetime.timedelta(days=i) for i in range(7)] # eg. [2022-04-11, 2022-04-12, ... ]

    # 3. append result to dates
    for date in date_int:
        dates.append(date)

# c. create dataframe and store results
df = pd.DataFrame(data = dates, columns = ['date'])
df['easter_week'] = 1 # our dummy for merging later
# df.to_pickle('./data/dates.pkl')

df


,date,easter_week
0,2000-04-17,1
1,2000-04-18,1
2,2000-04-19,1
3,2000-04-20,1
4,2000-04-21,1
...,...,...
156,2022-04-13,1
157,2022-04-14,1
158,2022-04-15,1
159,2022-04-16,1


## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

how to plot a series of dates only looking at months [stackoverflow](https://stackoverflow.com/questions/43283586/how-do-i-plot-a-histogram-of-months-with-dates-in-matplotlib)

In [168]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

df.dtypes

# df['dates'] = df['dates'].mask(df['dates'].dt.year == 2000, 
#                                 df['dates'] + pd.offsets.DateOffset(year=2000))

# dtm = lambda x: int(x[4:6])
# months = list(map(dtm, dates))

# fig, ax = plt.subplots()
# bins = np.arange(1,14)
# ax.hist(months, bins = bins, edgecolor="k", align='left')
# ax.set_xticks(bins[:-1])
# ax.set_xticklabels([datetime.date(1900,i,1).strftime('%b') for i in bins[:-1]] )

# plt.show()

dates          object
easter_week     int64
dtype: object

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.